In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from functools import reduce
from sklearn import preprocessing
from matplotlib import pyplot
from keras import models
from keras import layers
from keras import regularizers

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

%matplotlib inline


%matplotlib inline
%matplotlib inline

In [2]:
#Import OANDA Data
df_eurusd1 = pd.read_csv('Data_Investingcom/EUR_USD_1')
df_eurusd2 = pd.read_csv('Data_Investingcom/EUR_USD_2')
df_eurusd3 = pd.read_csv('Data_Investingcom/EUR_USD_3')

In [3]:
#print(df_eurusd1)
#print(df_eurusd2)
#print(df_eurusd3)

In [4]:
df_eurusd = pd.concat([df_eurusd1,df_eurusd2, df_eurusd3])
df_eurusd.index.name = 'number'
df_eurusd = df_eurusd.drop(['Open', 'Change %'], axis=1)
df_eurusd.head(20)

,Date,Price,High,Low
number,,,,
0,"Nov 21, 2018",1.1371,1.1410,1.1366
1,"Nov 20, 2018",1.1372,1.1472,1.1359
2,"Nov 19, 2018",1.1453,1.1467,1.1393
3,"Nov 16, 2018",1.1420,1.1423,1.1320
4,"Nov 15, 2018",1.1328,1.1364,1.1271
5,"Nov 14, 2018",1.1310,1.1349,1.1262
6,"Nov 13, 2018",1.1291,1.1296,1.1216
7,"Nov 12, 2018",1.1218,1.1333,1.1214
8,"Nov 09, 2018",1.1336,1.1371,1.1315


In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
df_eurusd_scaled = scaler.fit_transform(df_eurusd.values[:,1:])
df_eurusd.loc[:,1:] = df_eurusd_scaled
df_eurusd.head()

,Date,Price,High,Low
number,,,,
0,"Nov 21, 2018",0.516241,0.517561,0.522450
1,"Nov 20, 2018",0.516345,0.524023,0.521707
2,"Nov 19, 2018",0.524832,0.523502,0.525316
3,"Nov 16, 2018",0.521375,0.518916,0.517567
4,"Nov 15, 2018",0.511735,0.512767,0.512366


In [6]:
df_eurusd.to_csv('mergeddata.csv')

In [7]:
#step two

In [8]:
#Define function to add timesteps in datafram
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [9]:
#Add timesteps to the dataframe
dfm = pd.read_csv('mergeddata.csv', header=0, index_col=1)
values = dfm.values
n_days = 25
dfm = dfm.drop(['number'], axis = 1)
dfm.head()

,Price,High,Low
Date,,,
"Nov 21, 2018",0.516241,0.517561,0.522450
"Nov 20, 2018",0.516345,0.524023,0.521707
"Nov 19, 2018",0.524832,0.523502,0.525316
"Nov 16, 2018",0.521375,0.518916,0.517567
"Nov 15, 2018",0.511735,0.512767,0.512366


In [10]:
dfm = dfm.reindex(index=dfm.index[::-1])
dfm.head()

,Price,High,Low
Date,,,
"Dec 27, 1979",0.904233,0.899427,0.916039
"Dec 28, 1979",0.904443,0.899635,0.916251
"Dec 31, 1979",0.904966,0.900156,0.916782
"Jan 02, 1980",0.912091,0.907243,0.924000
"Jan 03, 1980",0.915025,0.910162,0.926972


In [11]:
dfm_series = series_to_supervised(dfm, n_days, 1)
dfm_series.head()

,var1(t-25),var2(t-25),var3(t-25),var1(t-24),var2(t-24),var3(t-24),var1(t-23),var2(t-23),var3(t-23),var1(t-22),...,var3(t-3),var1(t-2),var2(t-2),var3(t-2),var1(t-1),var2(t-1),var3(t-1),var1(t),var2(t),var3(t)
Date,,,,,,,,,,,,,,,,,,,,,
"Feb 01, 1980",0.904233,0.899427,0.916039,0.904443,0.899635,0.916251,0.904966,0.900156,0.916782,0.912091,...,0.905955,0.895013,0.890255,0.906698,0.890717,0.885982,0.902346,0.887992,0.883273,0.899586
"Feb 04, 1980",0.904443,0.899635,0.916251,0.904966,0.900156,0.916782,0.912091,0.907243,0.924000,0.915025,...,0.906698,0.890717,0.885982,0.902346,0.887992,0.883273,0.899586,0.888412,0.883689,0.900011
"Feb 05, 1980",0.904966,0.900156,0.916782,0.912091,0.907243,0.924000,0.915025,0.910162,0.926972,0.909996,...,0.902346,0.887992,0.883273,0.899586,0.888412,0.883689,0.900011,0.891450,0.886712,0.903089
"Feb 06, 1980",0.912091,0.907243,0.924000,0.915025,0.910162,0.926972,0.909996,0.905159,0.921877,0.914082,...,0.899586,0.888412,0.883689,0.900011,0.891450,0.886712,0.903089,0.895746,0.890985,0.907441
"Feb 07, 1980",0.915025,0.910162,0.926972,0.909996,0.905159,0.921877,0.914082,0.909224,0.926016,0.907795,...,0.900011,0.891450,0.886712,0.903089,0.895746,0.890985,0.907441,0.894070,0.889317,0.905742


In [12]:
#get var1(t) to the end
swapcol = list(dfm_series)
#print(swapcol)
swapcol[-3], swapcol[-1] = swapcol[-1], swapcol[-3]
dfm_series = dfm_series.loc[:,swapcol]

In [13]:
#dfm_series.index.name = 'Date'
dfm_series.to_csv('mergeddataseries.csv')
dfm_series.head()

,var1(t-25),var2(t-25),var3(t-25),var1(t-24),var2(t-24),var3(t-24),var1(t-23),var2(t-23),var3(t-23),var1(t-22),...,var3(t-3),var1(t-2),var2(t-2),var3(t-2),var1(t-1),var2(t-1),var3(t-1),var3(t),var2(t),var1(t)
Date,,,,,,,,,,,,,,,,,,,,,
"Feb 01, 1980",0.904233,0.899427,0.916039,0.904443,0.899635,0.916251,0.904966,0.900156,0.916782,0.912091,...,0.905955,0.895013,0.890255,0.906698,0.890717,0.885982,0.902346,0.899586,0.883273,0.887992
"Feb 04, 1980",0.904443,0.899635,0.916251,0.904966,0.900156,0.916782,0.912091,0.907243,0.924000,0.915025,...,0.906698,0.890717,0.885982,0.902346,0.887992,0.883273,0.899586,0.900011,0.883689,0.888412
"Feb 05, 1980",0.904966,0.900156,0.916782,0.912091,0.907243,0.924000,0.915025,0.910162,0.926972,0.909996,...,0.902346,0.887992,0.883273,0.899586,0.888412,0.883689,0.900011,0.903089,0.886712,0.891450
"Feb 06, 1980",0.912091,0.907243,0.924000,0.915025,0.910162,0.926972,0.909996,0.905159,0.921877,0.914082,...,0.899586,0.888412,0.883689,0.900011,0.891450,0.886712,0.903089,0.907441,0.890985,0.895746
"Feb 07, 1980",0.915025,0.910162,0.926972,0.909996,0.905159,0.921877,0.914082,0.909224,0.926016,0.907795,...,0.900011,0.891450,0.886712,0.903089,0.895746,0.890985,0.907441,0.905742,0.889317,0.894070


In [14]:
#step three

In [15]:
dfm_series = pd.read_csv('mergeddataseries.csv')
dfm_series = dfm_series.set_index('Date')
dfm_series.head()

,var1(t-25),var2(t-25),var3(t-25),var1(t-24),var2(t-24),var3(t-24),var1(t-23),var2(t-23),var3(t-23),var1(t-22),...,var3(t-3),var1(t-2),var2(t-2),var3(t-2),var1(t-1),var2(t-1),var3(t-1),var3(t),var2(t),var1(t)
Date,,,,,,,,,,,,,,,,,,,,,
"Feb 01, 1980",0.904233,0.899427,0.916039,0.904443,0.899635,0.916251,0.904966,0.900156,0.916782,0.912091,...,0.905955,0.895013,0.890255,0.906698,0.890717,0.885982,0.902346,0.899586,0.883273,0.887992
"Feb 04, 1980",0.904443,0.899635,0.916251,0.904966,0.900156,0.916782,0.912091,0.907243,0.924000,0.915025,...,0.906698,0.890717,0.885982,0.902346,0.887992,0.883273,0.899586,0.900011,0.883689,0.888412
"Feb 05, 1980",0.904966,0.900156,0.916782,0.912091,0.907243,0.924000,0.915025,0.910162,0.926972,0.909996,...,0.902346,0.887992,0.883273,0.899586,0.888412,0.883689,0.900011,0.903089,0.886712,0.891450
"Feb 06, 1980",0.912091,0.907243,0.924000,0.915025,0.910162,0.926972,0.909996,0.905159,0.921877,0.914082,...,0.899586,0.888412,0.883689,0.900011,0.891450,0.886712,0.903089,0.907441,0.890985,0.895746
"Feb 07, 1980",0.915025,0.910162,0.926972,0.909996,0.905159,0.921877,0.914082,0.909224,0.926016,0.907795,...,0.900011,0.891450,0.886712,0.903089,0.895746,0.890985,0.907441,0.905742,0.889317,0.894070


In [16]:
## create train and test split
n_features = 3
train = dfm_series.values[:9500, :]
test = dfm_series.values[9500:,:]

#split inputs and output
train_X, train_Y = train[:, :-3], train[:, -1]
test_X, test_Y = test[:, :-3], test[:, -1]

#make the train input 3 dimensional, idk why, think it's required for the lstm
train_X = train_X.reshape((train_X.shape[0], n_days, n_features))
test_X = test_X.reshape((test_X.shape[0], n_days, n_features))

print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape )

(9500, 25, 3) (9500,) (560, 25, 3) (560,)


In [17]:
#define model
def build_model():
    model = Sequential()
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2])))
    #model.add(LSTM(128))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

In [18]:
#kfold and fit model
k = 2
num_val_samples = len(train_X) // k
num_epochs = 128
size_batch = 64
all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_data = train_X[i * num_val_samples: (i + 1 ) * num_val_samples]
    val_targets = train_Y[i * num_val_samples: (i + 1 ) * num_val_samples]
    
    partial_train_data = np.concatenate(
    [train_X[:i *num_val_samples],
     train_X[(i +1) * num_val_samples: ]],
         axis =0)
    
    partial_train_targets = np.concatenate(
    [train_Y[:i*num_val_samples],
     train_Y[(i + 1) * num_val_samples:]],
         axis = 0)
    
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
             epochs= num_epochs, batch_size = size_batch, verbose = 0)
    
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose = 0)
    all_scores.append(val_mae)
    
#show the average mae
print(sum(all_scores)/len(all_scores))

processing fold # 0


UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/lstm/PartitionedCall]] [Op:__inference_train_function_2962]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
#Validate with test set
Y_hat = model.predict(test_X)
Y = test_Y

test_X = test_X.reshape((test_X.shape[0], n_days*n_features))


inv_Y_hat = concatenate((Y_hat, test_X[:, -2:]), axis=1)
inv_Y_hat = scaler.inverse_transform(inv_Y_hat)
inv_Y_hat = inv_Y_hat[:,0]

test_Y = test_Y.reshape((len(test_Y),1))
inv_Y = concatenate((test_Y, test_X[:, -2:]), axis=1)
inv_Y = scaler.inverse_transform(inv_Y)
inv_Y = inv_Y[:,0]



Y_hat_2 = model.predict(train_X)
Y_2 = train_Y

train_X = train_X.reshape((train_X.shape[0], n_days*n_features))
inv_Y_hat_2 = concatenate((Y_hat_2, train_X[:, -2:]), axis=1)
inv_Y_hat_2 = scaler.inverse_transform(inv_Y_hat_2)
inv_Y_hat_2 = inv_Y_hat_2[:,0]

train_Y = train_Y.reshape((len(train_Y),1))
inv_Y_2 = concatenate((train_Y, train_X[:, -2:]), axis=1)
inv_Y_2 = scaler.inverse_transform(inv_Y_2)
inv_Y_2 = inv_Y_2[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_Y, inv_Y_hat))
print('Train RMSE: %.3f' % rmse)

# calculate RMSE2
rmse2 = sqrt(mean_squared_error(inv_Y_2, inv_Y_hat_2))
print('Test RMSE2: %.3f' % rmse2)



In [ ]:
#plot actual stock price and predicted stock price
pyplot.figure(1)
pyplot.figure(figsize=(20,10))

pyplot.subplot(211)
pyplot.plot(inv_Y, label='actual')
pyplot.plot(inv_Y_hat, label='prediction')
pyplot.legend()

pyplot.subplot(212)
pyplot.plot(inv_Y_2, label='actual_withintraindata')
pyplot.plot(inv_Y_hat_2, label = 'prediction_withintraindata')
pyplot.legend()

pyplot.show()

In [ ]:
# Notes
# Still looking to add gridsearch for number of days, number of folds, batch size, number of epochs, 
# number of layers? number of neurons in LSTM layers.

# Want to add a plot where the model is used on another forex, to see wheter the model is just overfitted.

# Try the same procedure for a stock where also trade volume can be used (so not a forex).

# Pull the data through a sigmoid function to make it easier for the model to train.


